In [1]:
import pandas as pd
import numpy as np
from rsome import ro
from rsome import grb_solver as grb

## Data Ingestion

In [2]:
separation_time = pd.read_csv("data - Separation Time.csv",header=None).iloc[:10,:10].values
separation_time

array([[99999,     1,     3,     3,     3,     3,     3,     3,     3,
            3],
       [    1, 99999,     3,     3,     3,     3,     3,     3,     3,
            3],
       [    3,     3, 99999,     2,     2,     2,     2,     2,     2,
            2],
       [    3,     3,     2, 99999,     2,     2,     2,     2,     2,
            2],
       [    3,     3,     2,     2, 99999,     2,     2,     2,     2,
            2],
       [    3,     3,     2,     2,     2, 99999,     2,     2,     2,
            2],
       [    3,     3,     2,     2,     2,     2, 99999,     2,     2,
            2],
       [    3,     3,     2,     2,     2,     2,     2, 99999,     2,
            2],
       [    3,     3,     2,     2,     2,     2,     2,     2, 99999,
            2],
       [    3,     3,     2,     2,     2,     2,     2,     2,     2,
        99999]], dtype=int64)

In [3]:
landing_time_cost_df = pd.read_csv("data - Landing Time.csv").head(10)
landing_time_cost_df

,Airplane ID,Earliest Landing Slot,Target Landing Slot,Latest Landing Slot,Penalty cost per time slot of landing before target,Penalty cost per time slot of landing after target,Size
0,0,3,4,12,10,10,3
1,1,4,6,15,10,10,3
2,2,2,2,11,30,30,2
3,3,2,2,11,30,30,2
4,4,3,3,11,30,30,2
5,5,3,3,12,30,30,2
6,6,3,3,12,30,30,1
7,7,3,3,11,30,30,1
8,8,3,3,12,30,30,1
9,9,3,4,13,30,30,1


In [4]:
airplane_id = landing_time_cost_df["Airplane ID"].values
earliest_landing = landing_time_cost_df["Earliest Landing Slot"].values
target_landing = landing_time_cost_df["Target Landing Slot"].values
latest_landing = landing_time_cost_df["Latest Landing Slot"].values

early_landing_penalty = landing_time_cost_df["Penalty cost per time slot of landing before target"].values
late_landing_penalty = landing_time_cost_df["Penalty cost per time slot of landing after target"].values
airplace_size = landing_time_cost_df["Size"].values

In [5]:
runway_size = pd.read_csv("data - Runway.csv")['Max Size'].values
runway_size

array([3, 1, 2], dtype=int64)

# Model

In [6]:
def get_optimal_landing(airplane_id, separation_time, earliest_landing, target_landing, latest_landing, early_penalty, late_penalty,
                        runway_size, airplane_size):
    '''
    Prints the optimal landing time and runway allocation for each plane, among other variables.
    
    Parameters:
        airplane_id (numpy.ndarray): an array of the unique ID of each plane
        separation_time (numpy.ndarray): a matrix of separation times when planes land on the same runway
        earliest_landing (numpy.ndarray): an array of the earliest time each plane can land
        target_landing (numpy.ndarray): an array of the target landing time for each plane
        latest_landing (numpy.ndarray): an array of the latest time each plane can land
        early_penalty (numpy.ndarray): an array containing the penalty per unit time of landing each plane early
        late_penalty (numpy.ndarray): an array containing the penalty per unit time of landing each plane late
        runway_size (numpy.ndarray): an array of the size of the available runways
        airplane_size (numpy.ndarray): an array of the size of the available runways
    '''
    
    check_input_shapes(separation_time, len(airplane_id), len(earliest_landing), len(target_landing), len(latest_landing), 
                       len(early_penalty), len(late_penalty), len(airplane_size))
    
    no_of_planes = len(earliest_landing)
    no_of_runways = len(runway_size)
    max_airplace_runway_size_diff = max(airplane_size.max() -  runway_size.min(),  runway_size.max() - airplane_size.min())

    model = ro.Model('Airplane Landing')
    
    # Decision variables
    scheduled_landing = model.dvar(no_of_planes)
    earliness = model.dvar(no_of_planes)
    lateness = model.dvar(no_of_planes)
    lands_before = model.dvar((no_of_planes, no_of_planes), "B")
    lands_on_same_runway = model.dvar((no_of_planes, no_of_planes), "B")
    runway_allocation = model.dvar((no_of_planes, no_of_runways), "B")
    
    # Objective
    model.min(sum(early_penalty*earliness) + sum(late_penalty*lateness))
    
    # Constraints
    ## Airplane has to land between its latest and earliest possible time
    model.st(earliest_landing <= scheduled_landing)
    model.st(scheduled_landing <= latest_landing)

    ## Calculating the earliness & lateness of each scheduled landing
    model.st(scheduled_landing - target_landing == earliness - lateness)
    
    M = 99999
    for j in range(no_of_planes):
        for i in range(no_of_planes):
            if i != j:
                # If 2 planes are scheduled for the same runway, then at least X amt of time should have elapsed between plane i and plane j
                model.st(scheduled_landing[j] - scheduled_landing[i] >= 
                         separation_time[i,j]*lands_on_same_runway[i,j] - M*lands_before[j,i])
                
                # Ensure one airplane lands before the other
                model.st(lands_before[i,j] + lands_before[j,i] == 1)
                
                # Defining the link between lands_on_same_runway & runway_allocation
                for r in range(no_of_runways):
                    model.st(lands_on_same_runway[i,j] >= runway_allocation[i,r] + runway_allocation[j,r] - 1)
        
        # Each airplane can only land on 1 runway
        model.st(sum(runway_allocation[j,r] for r in range(no_of_runways)) == 1)

        # Ensures that airplane size should not be larger than size of runway
        for r in range(no_of_runways):
            model.st(airplane_size[j] - runway_size[r] <= (1 - runway_allocation[j,r])*max_airplace_runway_size_diff)


    model.st(scheduled_landing >= 0,
             earliness >= 0,
             lateness >= 0)
    
    # Get solution
    model.solve(grb)
    
    model_output = pd.DataFrame({
        'Airplace ID' : airplane_id,
        'Runway':  np.argmax(runway_allocation.get(), axis=1) + 1,
        'Scheduled Landing': scheduled_landing.get(),
        'Earliest Landing' : earliest_landing,
        'Latest Landing' : latest_landing,
        'Landing Earliness': earliness.get(),
        'Landing Lateness' : lateness.get()
        }).sort_values(['Runway','Scheduled Landing']).reset_index(drop=True)
    
    return model.get(), model_output

    
def check_input_shapes(separation, *args):
    assert len(set(args)) == 1, "Length of inputs must be equal"
    assert separation.shape == (args[0], args[0]), "Shape of separation must be NxN, where N is the length of each of the other inputs"

# Model Output

In [7]:
delay_cost, model_output = get_optimal_landing(airplane_id, separation_time, earliest_landing, target_landing, latest_landing, early_landing_penalty, late_landing_penalty, runway_size, airplace_size)

Restricted license - for non-production use only - expires 2024-10-28
Being solved by Gurobi...
Solution status: 2
Running time: 3.4350s


In [8]:
print(f'{delay_cost=}')
model_output

delay_cost=350.0


,Airplace ID,Runway,Scheduled Landing,Earliest Landing,Latest Landing,Landing Earliness,Landing Lateness
0,2,1,2.0,2,11,0.0,0.0
1,5,1,4.0,3,12,1.0,0.0
2,0,1,7.0,3,12,3.0,0.0
3,1,1,8.0,4,15,2.0,0.0
4,7,2,3.0,3,11,0.0,0.0
5,9,2,5.0,3,13,1.0,0.0
6,8,2,7.0,3,12,4.0,0.0
7,3,3,2.0,2,11,0.0,0.0
8,6,3,4.0,3,12,1.0,0.0
9,4,3,6.0,3,11,3.0,0.0
